In [20]:
import requests
import random
import json
import tqdm

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'

DATA    = {
    "message"   : "How do you treat peach leaf curl?",
    "sender"    : str(random.randint(0, 100000))
}

r = requests.post(URL, json = DATA)

json.loads(r.text)

[{'recipient_id': '76461',
  'text': 'Extracted slots</br>[Format: (<i>relation</i>) <strong>entity</strong> - <strong>value</strong>]:</br>Group 1:</br>(<i>pest</i>) <strong>descr</strong> - <strong>peach</strong></br>(<i>pest</i>) <strong>name</strong> - <strong>leaf curl</strong></br>(<i>pest</i>) <strong>action</strong> - <strong>treat</strong></br></br>Composed 1 additional queries:</br>1) <i>peach leaf curl treat</i></br>'},
 {'recipient_id': '76461',
  'text': 'Results with slots improvement... Top 9 results.'},
 {'recipient_id': '76461',
  'custom': {'text': 'Here are my top results:',
   'payload': 'collapsible',
   'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(score: 0.89)</br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>',
     'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development<

In [21]:
import random
import re

sender = random.randint(0, 100000)

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'
DATA    = {
    "message"   : "", 
    "sender"    : str(sender)
}
def run_tests(messages):
    results = []

    for m in tqdm.tqdm(messages):
        DATA['message'] = m
        r = requests.post(URL, json = DATA)
        r = json.loads(r.text)
        
        assert isinstance(r, list)
        assert len(r) >= 3
        r = r[2]

        assert 'custom' in r
        r = r['custom']

        assert 'data' in r
        r = r['data']

        result = []
        for e in r:
            title   = re.findall("<em>(.*?)</em>"           , e['title'])[0]
            link    = re.findall("href=[\"\'](.*?)[\"\']"   , e['title'])[0]
            result.append((title, link))
        
        results.append(result)

        DATA['message'] = '/intent_affirm'
        r = requests.post(URL, json = DATA)
        r = json.loads(r.text)

        assert len(r) == 1
        r = r[0]

        assert 'text' in r
        r = r['text']

        assert 'Anything else I can help with?' == r

    return results


In [30]:
import pandas as pd

FINAL_DATA = './data/transformed/result.pkl'

df = pd.read_pickle(FINAL_DATA)

df.sample(10)

,Question,ExpectedAnswer,URL,Source
21,Why is my tree dying?,"[Pest Notes: Wood Decay in Landscape Trees, Pe...",[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74109.ht...,UCIPM
50,"My milkweed have already bloomed, but now ther...",NaN,[http://ipm.ucanr.edu/PMG/r606301211.html],AE
44,How do I deter deer from my garden?,[Pest Notes: Deer],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74117.html],UCIPM
46,An animal is eating all the plants in my garde...,[Pest Notes: Deer],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74117.html],UCIPM
47,How can we control the fruit flies we have in ...,NaN,[https://www2.ipm.ucanr.edu/agriculture/canebe...,AE
17,How did I get carpet beetles in my home?,[Pest Notes: Carpet Beetles],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7436.html],UCIPM
16,Where are the little beetles on my window sill...,[Pest Notes: Carpet Beetles],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7436.html],UCIPM
48,"I have flying bugs, like over-sized fruit flie...",NaN,[http://ipm.ucanr.edu/PMG/invertebrates/links....,AE
39,I have a problem with fruit flies laying their...,NaN,[https://www2.ipm.ucanr.edu/agriculture/canebe...,AE
3,I started noticing some sort of bug that made ...,NaN,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7420.htm...,AE


In [ ]:
df[['Question', 'UR']]

In [23]:
answers     = df['URL'].values.tolist()
results     = run_tests(df['Question'])

100%|██████████| 97/97 [02:30<00:00,  1.55s/it]


In [24]:
scores = []
for i, r in enumerate(results):
    answer = answers[i]
    topn = [False, False, False, False]
    for i1, r1 in enumerate(r):
        if r1[1].split('?')[0] in answer:
            if i1 == 0:
                topn[0] = True
            if i1 < 3:
                topn[1] = True
            if i1 < 5:
                topn[2] = True
            topn[3] = True
    scores.append(topn)

In [29]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 27  (27.84%)
Top 3 : 34  (35.05%)
Top 5 : 38  (39.18%)
Top 10: 43  (44.33%)
